In [1]:
import tensorflow as tf
import numpy as np

2022-04-12 08:32:10.883651: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-12 08:32:10.883674: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


`x_train` og `x_test` er begge to billeder af håndskrevne tal. `y_test` og `y_train` er de rigtige værdier for hvert tal

forskellen på train og test er at train dataen skal bruges mens det neurale netværk lærer og test bliver brugt til at evaluere hvor godt netværket er. Siden netværket aldrig har set disse eksempler før så kan dette vise hvor godt netværket fungere på ukendt data uden at den kan overfitte.

In [3]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Nu skal netværket defineres, det her er altså hvilke lag dataen skal igennem, her skal man også definere hvordan input data ser ud

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

2022-04-12 08:32:13.497939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-12 08:32:13.497968: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-12 08:32:13.497989: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-P6CV8G1): /proc/driver/nvidia/version does not exist
2022-04-12 08:32:13.498230: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Her efter prøver vi at kører modellen vi lige har defineret, på det første billede i vores trænings data. outputtet herfra er vores 10 nodes som vi definerede i vores netværk struktur set over

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.26483682, -0.07163045,  0.01862767,  0.69676214,  0.33395773,
        -0.7474797 ,  0.26209992, -0.00665359, -0.03824785, -0.99384093]],
      dtype=float32)

Det her data er mest af alt ubrugeligt siden netværket ikke er trænet endnu og derfor bare spytter tilfældige værdier ud. Men hvert tal her svarer til hvad netværket tror tallet er fra 0-9

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.12116387, 0.08654606, 0.09472091, 0.18661903, 0.12983505,
        0.04402808, 0.12083271, 0.09235626, 0.08948395, 0.0344141 ]],
      dtype=float32)

Vi kan bruge softmax til at lave disse tal om til en procent der svarer til hvor stor procent netværket tror det er svaret

Nu skal der vælges en loss funktion. Her bruger vi SparseCategoricalCrossentropy siden vi har flere kategorier, hvis vi havde 2 valg sandt eller falsk ville man nok bruge binarycrossentropy 

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Nu hvor vi har en loss fn så kan vi prøve at teste de tidligere predictions med mærkattet fra data sættet.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

3.1229277

Nu har vi prøvet det hele af, så nu skal modellen bare trænes. Her kan vælge i mellem mange forskellige algoritmer, her burger vi bare adam. Metrics skal bare være accuracy med mindre man gør noget mærkeligt

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Så starter vi trænningen med vores trænings data

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2985 - accuracy: 0.9131
Epoch 2/5
   1/1875 [..............................] - ETA: 6s - loss: 0.0620 - accuracy: 1.0000

KeyboardInterrupt: 

Nu hvor modellen er trænet kan vi finde ud af hvor god den er

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0769 - accuracy: 0.9759 - 322ms/epoch - 1ms/step


[0.07687602192163467, 0.9758999943733215]

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.0338330e-07, 5.1836340e-09, 6.5386293e-06, 1.3139765e-04,
        1.8055530e-12, 1.1917374e-06, 4.9531180e-14, 9.9985814e-01,
        6.2987408e-07, 2.0451096e-06],
       [2.6960418e-09, 8.5740787e-05, 9.9990129e-01, 1.1792849e-05,
        1.9633228e-13, 1.0450751e-06, 2.3839775e-09, 3.1142008e-13,
        1.4055037e-07, 4.6210038e-14],
       [1.0778501e-06, 9.9758279e-01, 2.0820716e-04, 1.3445839e-04,
        1.0964774e-05, 2.6997212e-05, 5.8267919e-06, 1.4382901e-03,
        5.8356812e-04, 7.8728599e-06],
       [9.9980670e-01, 1.2216687e-11, 6.0954961e-05, 1.3395494e-07,
        2.7682168e-07, 1.5158237e-05, 1.1403263e-04, 9.0104908e-07,
        1.3221576e-09, 1.9540662e-06],
       [6.2404126e-07, 4.4980226e-08, 1.3313356e-05, 5.9294490e-07,
        9.9741024e-01, 4.2079762e-08, 9.3958664e-07, 9.2096889e-05,
        1.5024169e-07, 2.4820683e-03]], dtype=float32)>